<a href="https://colab.research.google.com/github/nehabindle/Yelp-Review-Classification/blob/master/YelpNeha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s gdrive/'Team Drives'/'Data Mining Team'/ gdata

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!ls gdata/

 LogisticReg.ipynb	    tokenizer.ipynb
 lr.ipynb		    Untitled0.ipynb
 ModifiedData.pkl	   'Untitled document.gdoc'
 my_model.h5		    yelpDataset.ipynb
 Neha_SVM.ipynb		    yelpDataset_kunj2.ipynb
'Preprocessing link.gdoc'   yelpDataset_kunj.ipynb
'Project proposal.gdoc'     yelpDataset_shrey.ipynb
 shrey_yelp.ipynb	    yelpHotelData.db
 tokenized_X_combine.pkl    YelpNeha.ipynb
 tokenized_X_test.pkl	    Yelp_Neha_TensorFlow.ipynb
 tokenized_X_train.pkl


In [3]:
!pip install contractions

In [0]:
import numpy as np
import pandas as pd
import nltk
import spacy
from tqdm import tqdm
import os
import seaborn as sns
import matplotlib.pyplot as plt
import re, string, unicodedata
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

In [0]:
import sqlite3
import pandas as pd
connection = sqlite3.connect('gdata/yelpHotelData.db')
x1 = connection.execute("select * FROM review")

In [0]:
description = []
Class = []

data = x1.fetchmany(10000)

for x in data:
  description.append(x[3])
  Class.append(x[8])
trainData = {'Class' : Class, 'Description' : description}
df_X = pd.DataFrame(trainData)

In [7]:
df_X['Class'].value_counts()
#df_X

N     5078
NR    2461
YR    1681
Y      780
Name: Class, dtype: int64

In [0]:
import time
import pickle
pd.set_option('display.max_colwidth', 200)

In [9]:
df_X['Class'].value_counts(normalize = True)

N     0.5078
NR    0.2461
YR    0.1681
Y     0.0780
Name: Class, dtype: float64

In [0]:
# remove URL's from train and test
 df_X['clean_desc']= df_X['Description'].apply(lambda x: re.sub(r'http\S+', '', x))


In [0]:
# remove punctuation marks
punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~.,'

df_X['clean_desc'] = df_X['clean_desc'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))

In [0]:
# convert text to lowercase
df_X['clean_desc'] = df_X['clean_desc'].str.lower()

In [0]:
# remove whitespaces
df_X['clean_desc'] = df_X['clean_desc'].apply(lambda x:' '.join(x.split()))

In [0]:
# import spaCy's language model
nlp = spacy.load('en', disable=['parser', 'ner'])


In [0]:
# tokenization using spaCy
def tokenization(x):
    desc_tokens = []
    for i in tqdm(x):
        i = nlp(i)
        temp = []
        for j in i:
            temp.append(j.text)
        desc_tokens.append(temp)
    
    return desc_tokens

In [0]:
#Tokenization for Train data
df_X['clean_desc'] = tokenization(df_X['clean_desc'])

 44%|████▍     | 4437/10000 [00:43<00:53, 103.46it/s]

In [0]:
#function to remove stopwords
def remove_stopwords(desc):
    s = []
    for r in tqdm(desc):
        s_2 = []
        for token in r:
            if nlp.vocab[token].is_stop == True:
                continue
            else:
                s_2.append(token)
        s.append(" ".join(s_2))    
        
    return s

In [0]:
df_X['clean_desc'] = remove_stopwords(df_X['clean_desc'])

In [0]:
# function to lemmatize text
def lemmatization(texts):
    output = []
    for i in texts:
             s = [token.lemma_ for token in nlp(i)]
             output.append(' '.join(s))
    return output

In [0]:
df_X['clean_desc'] = lemmatization(df_X['clean_desc'])

In [0]:
df_X.sample(10)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC

In [0]:
#preparing the final pipeline using the selected parameters
model = Pipeline([('vectorizer', CountVectorizer(ngram_range=(1,2))),
    ('tfidf', TfidfTransformer(use_idf=True)),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight="balanced")))])

#the class_weight="balanced" option tries to remove the biasedness of model towards majority sample

In [0]:
from sklearn.model_selection import learning_curve, GridSearchCV
parameters = {'vectorizer__ngram_range': [(1, 1), (1, 2),(2,2)],
               'tfidf__use_idf': (True, False)}
gs_clf_svm = GridSearchCV(model, parameters, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(df_X['clean_desc'],df_X['Class'] )
print(gs_clf_svm.best_score_)
print(gs_clf_svm.best_params_)

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
# from sklearn.feature_extraction.text import TfidfVectorizer


In [0]:
# # build TF-IDF features for train data
# cv = TfidfVectorizer(use_idf=True, min_df=5, max_df=0.4, ngram_range=(1,2),
#                         sublinear_tf=True,)
# cv_train = cv.fit_transform(df_X['clean_desc'])

In [0]:
# cv_train

Training the model using SVM

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df_X['clean_desc'],df_X['Class'], test_size=0.3, random_state=42)

In [0]:
X_train.shape

In [0]:
X_test.shape

In [0]:
y_train.shape

In [0]:
y_test.shape

In [0]:
#fit model with training data
model.fit(X_train, y_train)
#evaluation on test data
pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(pred, y_test)r

In [0]:
from sklearn import metrics
print(metrics.classification_report(y_test, pred))

In [0]:
from sklearn import svm

In [0]:
clf = svm.SVC(gamma='scale', decision_function_shape='ovo')

clf.fit(X_train,y_train)print(metrics.classification_report(y_test, y_pred))

In [0]:
from sklearn import metrics
y_pred = clf.predict(X_test)
print(metrics.classification_report(y_test, y_pred))